In [23]:
import urllib.request # Para leer xml de los monumentos y edificios
import requests
import xml.sax
import re
import math
from bottle import route, run, request, template

### Creación de clases necesarias

In [2]:
class Edificio:
    def __init__(self):
        self.idEntidad = ""
        self.nombre = ""
        self.descripcion = ""
        self.horario = ""
        self.equipamiento = ""
        self.transporte = ""
        self.accesibilidad = ""
        self.url = ""
        
        #Campos localizacion
        self.nombreVia = ""
        self.claseVia = ""
        self.numero = ""
        self.localidad = ""
        self.provincia = ""
        self.codigoPostal = ""
        self.barrio = ""
        self.distrito = ""
        self.coordenadaY = ""
        self.coordenadaX = ""
        
        # Datos de contacto
        self.telefono = ""
        self.email = ""
        
        self.tipo = ""
        
    def __str__(self):
        cadena = ""
        cadena += "Id de la entidad: " + self.idEntidad + "\n"
        cadena += "Nombre: " + self.nombre + "\n"
        cadena += "Descripcion: " + self.descripcion + "\n"
        cadena += "Horario: " + self.horario + "\n"
        cadena += "Equipamiento: " + self.equipamiento + "\n"
        cadena += "Transporte: " + self.transporte + "\n"
        cadena += "Accesibilidad: " + self.accesibilidad + "\n"
        cadena += "URL: " + self.url + "\n"
        cadena += "Direccion: " + str(self.claseVia) + " " + str(self.nombreVia) + "\n"
        cadena += "Numero: " + self.numero + "\n"
        cadena += "Localidad: " + self.localidad + "\n"
        cadena += "Provincia: " + self.provincia + "\n"
        cadena += "Codigo postal: " + self.codigoPostal + "\n"
        cadena += "Barrio: " + self.barrio + "\n"
        cadena += "Distrito: " + self.distrito + "\n"
        cadena += "Latitud: " + self.coordenadaY + "\n"
        cadena += "Longitud: " + self.coordenadaX + "\n"
        cadena += "Telefono: " + self.telefono + "\n"
        
        if self.email != "":
            cadena += "Email: " + self.email + "\n"
            
        return cadena

In [3]:
class ManejadorCatalogo (xml.sax.ContentHandler):
    
    def __init__(self):
        self.Datos = ""
        self.edificios = []
        self.edificio = Edificio()

    def startElement(self, etiqueta, atributos):
        if etiqueta == "atributo":
            self.Datos = atributos["nombre"]
            
    def endElement(self, etiqueta):
        if etiqueta == "contenido":
            self.edificios.append(self.edificio)
            self.edificio = Edificio()
        self.Datos = ""
            
    def characters(self, contenido):
        if self.Datos == "NOMBRE":
            self.edificio.nombre = contenido
        elif self.Datos == "ID-ENTIDAD":
            self.edificio.idEntidad = contenido
        elif self.Datos == "DESCRIPCION-ENTIDAD":
            self.edificio.descripcion = contenido
        elif self.Datos == "HORARIO":
            self.edificio.horario = contenido
        elif self.Datos == "EQUIPAMIENTO":
            self.edificio.equipamiento = contenido
        elif self.Datos == "TRANSPORTE":
            self.edificio.transporte = contenido
        elif self.Datos == "ACCESIBILIDAD":
            self.edificio.accesiblidad = contenido
        elif self.Datos == "CONTENT-URL":
            self.edificio.url = contenido
        elif self.Datos == "NOMBRE-VIA":
            self.edificio.nombreVia = contenido
        elif self.Datos == "CLASE-VIAL":
            self.edificio.claseVia = contenido
        elif self.Datos == "NUM":
            self.edificio.numero = contenido
        elif self.Datos == "LOCALIDAD":
            self.edificio.localidad = contenido
        elif self.Datos == "PROVINCIA":
            self.edificio.provincia = contenido
        elif self.Datos == "CODIGO-POSTAL":
            self.edificio.codigoPostal = contenido
        elif self.Datos == "BARRIO":
            self.edificio.barrio = contenido
        elif self.Datos == "DISTRITO":
            self.edificio.distrito = contenido
        elif self.Datos == "COORDENADA-X":
            self.edificio.coordenadaX = contenido
        elif self.Datos == "COORDENADA-Y":
            self.edificio.coordenadaY = contenido
        elif self.Datos == "LATITUD":
            self.edificio.coordenadaY = contenido
        elif self.Datos == "LONGITUD":
            self.edificio.coordenadaX = contenido
        elif self.Datos == "TELEFONO":
            self.edificio.telefono = contenido
        elif self.Datos == "EMAIL":
            self.edificio.email = contenido
        elif self.Datos == "TIPO":
            self.edificio.tipo = contenido

In [4]:
class ManejadorCoordenadas (xml.sax.ContentHandler):
    
    def __init__(self):
        self.Datos = ""
        self.coordenadaX = ""
        self.coordenadaY = ""
        self.primerasCoordenadasEncontradas = True

    def startElement(self, etiqueta, atributos):
        if etiqueta == "CoordinateX":
            self.Datos = "CoordinateX"
        elif etiqueta == "CoordinateY":
            self.Datos = "CoordinateY"
            
    def endElement(self, etiqueta):
        if self.primerasCoordenadasEncontradas and self.coordenadaX != "" and self.coordenadaY != "":
            self.primerasCoordenadasEncontradas = False
            
        self.Datos = ""
            
    def characters(self, contenido):
        if self.primerasCoordenadasEncontradas:
            if self.Datos == "CoordinateX":
                self.coordenadaX = contenido
            elif self.Datos == "CoordinateY":
                self.coordenadaY = contenido

In [5]:
class ManejadorRuta (xml.sax.ContentHandler):
    
    def __init__(self):
        self.Datos = ""
        self.fecha = ""
        self.horaInicio = ""
        self.horaEstimadaLlegada = ""
        self.numeroDeTrasbordos = ""
        self.duracion = ""
        self.descripcionTextual = []

    def startElement(self, etiqueta, atributos):
        if etiqueta == "DescriptionDate":
            self.Datos = "fecha"
        elif etiqueta == "DescriptionInitTime":
            self.Datos = "horaInicio"
        elif etiqueta == "DescriptionEstimateTimeArrival":
            self.Datos = "horaEstimadaLlegada"
        elif etiqueta == "Transfers":
            self.Datos = "numeroDeTrasbordos"
        elif etiqueta == "LongJourney":
            self.Datos = "duracion"
        elif etiqueta == "RouteDescription":
            self.Datos = "descripcionDeLaRuta"
            
    def endElement(self, etiqueta):
        self.Datos = ""
            
    def characters(self, contenido):
        if self.Datos == "fecha":
            self.fecha = contenido
        elif self.Datos == "horaInicio":
            self.horaInicio = contenido
        elif self.Datos == "horaEstimadaLlegada":
            self.horaEstimadaLlegada = contenido
        elif self.Datos == "numeroDeTrasbordos":
            self.numeroDeTrasbordos = contenido
        elif self.Datos == "duracion":
            self.duracion = contenido
        elif self.Datos == "descripcionDeLaRuta":
            self.descripcionTextual.append(contenido)

### Descarga y parseo del XML que contiene edificios y monumentos

In [9]:
def descargarCatalogo():
    x = urllib.request.urlopen('https://datos.madrid.es/egob/catalogo/208844-0-monumentos-edificios.xml')
    a = x.read()
    a = a.decode("utf-8")
    prueba = open("catalogo.xml","w")
    prueba.write(str(a))
    prueba.close()

In [10]:
def parsearCatalogo():
    parser = xml.sax.make_parser()
    parser.setFeature(xml.sax.handler.feature_namespaces,0)
    manejadorCatalogo = ManejadorCatalogo()
    parser.setContentHandler(manejadorCatalogo)
    parser.parse("catalogo.xml")
    return manejadorCatalogo.edificios

In [13]:
descargarCatalogo()

In [14]:
edificios = parsearCatalogo()

SAXParseException: catalogo.xml:15:32: not well-formed (invalid token)

### Acceso a los servicios EMT

In [15]:
idCliente = 'WEB.SERV.semart12@ucm.es'
passKey = '1CF11EBE-019D-4BDB-80AF-D9814F4B1E85' 

In [16]:
def obtenerCoordenadasDe(calle, numero):
    datos = {
        'idClient': idCliente,
        'PassKey': passKey,
        'description': calle,
        'streetNumber': str(numero),
        'Radius':'',
        'Stops':'',
        'statistics':'',
        'cultureInfo':''
    }
 
    url = 'https://servicios.emtmadrid.es:8443/geo/servicegeo.asmx/GetStreet'
    response = requests.post(url, data=datos)
    handler = ManejadorCoordenadas()
    xml.sax.parseString(response.text, handler)
    
    return handler.coordenadaX, handler.coordenadaY

In [17]:
def calcularRuta(coordenadaXOrigen, coordenadaYOrigen, coordenadaXDestino, coordenadaYDestino):
    datos = {
        'idClient': idCliente,
        'PassKey': passKey,
        'statistics':'',
        'cultureInfo':'',
        'coordinateXFrom': coordenadaXOrigen,
        'coordinateYFrom': coordenadaYOrigen,
        'originName':'',
        'coordinateXTo': coordenadaXDestino,
        'coordinateYTo': coordenadaYDestino,
        'destinationName':'',
        'criteriaSelection':'11',
        'day':'',
        'month':'',
        'year':'',
        'hour':'',
        'minute':'',
        'GenerarAudio':''
    }
 
    url = 'https://servicios.emtmadrid.es:8443/servicemedia/servicemedia.asmx/GetStreetRoute'
    response = requests.post(url, data=datos)
    handler = ManejadorRuta()
    xml.sax.parseString(response.text, handler)
    return handler

### Funciones auxiliares

In [18]:
def applyRegex(cadena):
    # Para que la fecha y la hora de inicio del itinerario se muestren bien
    m = re.search('\w+:(\d.+$)', cadena)
    return m.group(1)

In [19]:
# Latitud = coordenada Y 
# Longitud = coordenada X
def calculoDistanciaHaversine(lat1, lon1, lat2, lon2):
    rad = math.pi / 180
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    R = 6372.795477598
    a = (math.sin(rad*dlat/2))**2 + math.cos(rad*lat1)*math.cos(rad*lat2)*(math.sin(rad*dlon/2))**2
    distancia = 2 * R * math.asin(math.sqrt(a))
    return distancia

In [20]:
def obtenerEdificiosCercanos(coordenadaXOrigen, coordenadaYOrigen, distanciaMaxima):
    edificiosCercanos = []
    
    for edificio in edificios:    
        if float(distanciaMaxima) >= calculoDistanciaHaversine(float(coordenadaYOrigen), float(coordenadaXOrigen), float(edificio.coordenadaY), float(edificio.coordenadaX)):
            edificiosCercanos.append(edificio)
    
    return edificiosCercanos

### Mostrar al usuario los 3 servicios

In [21]:
@route ('/')
def index():
    return template("index")

### Rutas del servicio 1

In [22]:
@route ('/mostrar_museos')
def mostrarMuseos():
    return template("museos", edificios=edificios)

In [ ]:
@route ('/mostrar_museo_seleccionado', method='POST')
def mostrarMuseoSeleccionado():
    indiceDelMuseo = request.forms.get('museo')
    return template("mostrar_museo", museo=edificios[int(indiceDelMuseo) - 1])

### Rutas del servicio 2

In [ ]:
@route ('/obtener_datos_calculo_itinerario')
def obtencionDeDatosParaElCalculoDelItinerarioOptimo():
    return template("obtener_datos_calculo_itinerario", edificios=edificios)

In [ ]:
@route ('/mostrar_itinerario', method='POST')
def mostrarItinerarioOptimo():
    calle = request.forms.get('calle')
    numero = request.forms.get('numero')
    indiceDelEdificio = request.forms.get('edificio')
    
    coordenadaXOrigen, coordenadaYOrigen = obtenerCoordenadasDe(calle, numero)
    edificio = edificios[int(indiceDelEdificio) - 1]
    ruta = calcularRuta(coordenadaXOrigen, coordenadaYOrigen, edificio.coordenadaX, edificio.coordenadaY)
    ruta.fecha = applyRegex(ruta.fecha)
    ruta.horaInicio = applyRegex(ruta.horaInicio)
        
    return template("mostrar_itinerario_optimo", ruta=ruta)

### Rutas del servicio 3

In [ ]:
@route ('/obtener_datos_museos_cercanos')
def obtenerDatosDeMuseosCercanos():
    return template("obtener_datos_museos_cercanos")

In [ ]:
@route ('/mostrar_edificios_cercanos', method='POST')
def mostrarMuseosCercanos():
    calle = request.forms.get('calle')
    numero = request.forms.get('numero')
    distanciaMaxima = request.forms.get('distancia')
    
    coordenadaXOrigen, coordenadaYOrigen = obtenerCoordenadasDe(calle, numero)
    edificiosCercanos = obtenerEdificiosCercanos(coordenadaXOrigen, coordenadaYOrigen, distanciaMaxima)
    return template("mostrar_edificios_cercanos", edificios=edificiosCercanos)

In [ ]:
run(host='0.0.0.0', port=8080)

Bottle v0.12.13 server starting up (using WSGIRefServer())...
Listening on http://0.0.0.0:8080/
Hit Ctrl-C to quit.

127.0.0.1 - - [01/Nov/2018 19:04:55] "GET / HTTP/1.1" 200 1592
127.0.0.1 - - [01/Nov/2018 19:04:55] "GET /static/style/style.css HTTP/1.1" 404 762
127.0.0.1 - - [01/Nov/2018 19:04:56] "GET /static/style/style.css HTTP/1.1" 404 762
127.0.0.1 - - [01/Nov/2018 19:04:56] "GET /mostrar_museos HTTP/1.1" 200 9421
127.0.0.1 - - [01/Nov/2018 19:04:57] "GET /static/style/style.css HTTP/1.1" 404 762
127.0.0.1 - - [01/Nov/2018 19:04:57] "GET / HTTP/1.1" 200 1592
127.0.0.1 - - [01/Nov/2018 19:04:57] "GET /obtener_datos_calculo_itinerario HTTP/1.1" 200 10046
127.0.0.1 - - [01/Nov/2018 19:04:57] "GET /static/style/style.css HTTP/1.1" 404 762
127.0.0.1 - - [01/Nov/2018 19:04:57] "GET /favicon.ico HTTP/1.1" 404 740
127.0.0.1 - - [01/Nov/2018 19:04:58] "GET / HTTP/1.1" 200 1592
127.0.0.1 - - [01/Nov/2018 19:04:58] "GET /static/style/style.css HTTP/1.1" 404 762
127.0.0.1 - - [01/Nov/2018 1